In [56]:
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
import json

In [57]:
client_id=''
client_secret=''
redirect_uri='http://google.com/'


In [58]:
username='n.acar'
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret) 
scope = 'user-library-read playlist-read-private'
try:
    token = util.prompt_for_user_token(username, scope,client_id=client_id, client_secret=client_secret, redirect_uri=redirect_uri)  
    sp=spotipy.Spotify(auth= token)
except:
    print('Token is not accesible for ' + username)



In [59]:
#taking the songs library
results = sp.current_user_saved_tracks()
for item in results['items']:
    track = item['track']
    print(track['id'])
    

3axhnTsNa4YefoyLGxjdWz
0MMNWxGJ4SkZVmajWv6MxX
5nNmj1cLH3r4aA4XDJ2bgY
2KA6dhXmz72kvaEK0liYZL
6A6qBFJHid4WutQu6HHEZt
6n2Hr4ieid3D9ajnGX3pdp
1DoS5ql0TW3ejjJGHbsAmz
3UVj0QFwXzsvVnZFh8ABNJ
64BbRFQBFPpSKhmCZ5w7K1
4DcJxTf3hXEaJIN06xKFb8
0mDDfA4yNuyw32ETB38jBj
7nnW9qKcLQi98ghW42lsAX
5gGvmMgnq66xLSEoDx9iJQ
2pD1BR7vPz8GiygzRKmg5n
72b3Ws3byP6yidFilsOoyz
72UMOAfc73lO9ci7Fzl66X
5i9YJPhJo1RrWHKjkbkgmk
1XhFmIIYWjHuovs2F4IpWj
3eZGNBvKkExXp8q5FAyM8y
2EbQzx4SPYv6Aup9mhP9U0


In [62]:
#getting the audio features of a song
results2 = sp.audio_features(tracks=['3axhnTsNa4YefoyLGxjdWz'])
print(results2)

[{'danceability': 0.8, 'energy': 0.491, 'key': 10, 'loudness': -8.293, 'mode': 0, 'speechiness': 0.0468, 'acousticness': 0.309, 'instrumentalness': 0, 'liveness': 0.332, 'valence': 0.425, 'tempo': 135.008, 'type': 'audio_features', 'id': '3axhnTsNa4YefoyLGxjdWz', 'uri': 'spotify:track:3axhnTsNa4YefoyLGxjdWz', 'track_href': 'https://api.spotify.com/v1/tracks/3axhnTsNa4YefoyLGxjdWz', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/3axhnTsNa4YefoyLGxjdWz', 'duration_ms': 187654, 'time_signature': 4}]
